In [26]:
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import pandas as pd
from pandas import DataFrame as df

import re
import csv

dask.config.set(scheduler='multiprocessing')
dask.config.set({"array.slicing.split_large_chunks": True})

### Combine 3 Datasets

In [ ]:
'''
TODO:
    Function to efficiently search for answer code block values
    Execute code blocks (ANTLR)
'''

# Regex to extract code blocks
* Find max length block and transfer to new column
* Check for "ERROR" and drop columns

## Parse main code block

In [2]:
def extract_code_block(body) -> str:
    pattern = r'<code>(.*?)</code>'
    match = re.findall(pattern, body)
    return "Error" if not match else str(max(match, key=len))

### Testing

In [3]:
string1 = "Testing <code>long</code> something about this block <code>longer</code> xyz <code>longest</code>"
string2 = "Test without code"

In [4]:
df = pd.DataFrame()
df['body'] = [string1, string2]

In [5]:
df['block'] = df['body'].map(lambda x : extract_code_block(x))

In [6]:
df

,body,block
0,Testing <code>long</code> something about this...,longest
1,Test without code,Error


## Parse Body

In [3]:
def extract_body_block(body) -> str:
    pattern = r'<p>(.*?)</p>'
    code = r'(<.*?>)'
    match = re.findall(pattern, body)
    if not match:
        return "Error"
    else:
        match = [re.sub(code, "", m) for m in match]
        strings = [str(m).lower() for m in match]
        return '\n'.join(strings)

## Structure of x, y pre-clean

* Parse body and code for all rows
* drop all columns apart from id, accepted id and text
* pair code block of accepted with body of original into new frame

## Cleaning

* Compile code blocks?
* Clean y data, maybe remove contractions?
* Maybe generate new pairs to enrich language?



In [4]:
def to_pandas(path) -> pd.DataFrame:
    tmp = dd.read_parquet(path, engine='fastparquet')  
    print("Converting to Pandas Frame...")
    with ProgressBar():
        df = tmp.compute()
        df.reset_index()
        print("Done")
    return df

In [22]:
def parse_body_blocks(path) -> pd.DataFrame:
    tmp = dd.read_parquet(path, engine='fastparquet')  

    print("Converting to Pandas Frame...")
    with ProgressBar():
        df = tmp.compute()
        df.reset_index()
        print("Done")

    print("Extracting body text...")
    with ProgressBar():
        df['text'] = df['body'].map(lambda x : extract_body_block(x))
        print("Done")
    
    df.drop(['id', 'score', 'title', 'tags', 'body', 'post_type_id'], axis='columns', inplace=True)
    df = remove_errors(df, 'text')

    return df

In [15]:
def remove_errors(frame, type) -> pd.DataFrame:
    print("Removing empty {} blocks...".format(type))
    frame = frame[frame[type] != "Error"]
    print("Done")
    return frame

In [29]:
def lookup(x, val) -> str:
    row = x.query("id == {}".format(val))
    return "Error" if not row else row["body"]

def pairs(frame, y_path) -> pd.DataFrame:
    df = pd.read_csv(y_path, engine='python', encoding='utf-8')
    df = dd.from_pandas(df)

    print("Lookup on answers...")
    with ProgressBar():
        df['code'] = df['accepted_answer_id'].map(lambda x : lookup(frame, x)).compute()

    df['code'] = df['body'].map(lambda x : extract_code_block(x))
    df.drop(['body'], axis='columns', inplace=True)
    df = remove_errors(df, 'code')

    print("Converting to Pandas Frame...")
    with ProgressBar():
        df = df.compute()
        print("Done")

    return df

In [8]:
def prep_y(path, out) -> None:
    tmp = parse_body_blocks(path)
    tmp.to_csv(out)

## Preparation

### Each language set is parsed and stored in a new csv

In [9]:
prep_y('D:\PROJECT\dataset_select\python.parq', 'D:\PROJECT\dataset_select\y\python.csv')
prep_y('D:\PROJECT\dataset_select\java.parq', 'D:\PROJECT\dataset_select\y\java.csv')
prep_y('D:\PROJECT\dataset_select\c.parq', 'D:\PROJECT\dataset_select\y\c.csv')

Converting to Pandas Frame...
[########################################] | 100% Completed | 12.8s
Done
Extracting body text...
Done
Removing empty text blocks...
Done
Converting to Pandas Frame...
[########################################] | 100% Completed | 27.2s
Done
Extracting body text...
Done
Removing empty text blocks...
Done
Converting to Pandas Frame...
[########################################] | 100% Completed |  9.4s
Done
Extracting body text...
Done
Removing empty text blocks...
Done


### Creating new parquet with only answers

* Post type 2 corresponds to answers
* Other columns are dropped to save space during mapping

In [10]:
posts = dd.read_parquet('D:\PROJECT\dataset_parq\posts.parq', engine='fastparquet')

In [12]:
answers = posts.query("post_type_id == 2")
answers = answers.drop(['accepted_answer_id', 'score', 'title', 'tags', 'post_type_id'], axis='columns')

In [14]:
with ProgressBar():
    answers.repartition(npartitions=100)\
        .to_parquet('D:\PROJECT\dataset_parq/answers.parq', engine='fastparquet', compression='snappy')

[########################################] | 100% Completed |  1min 32.3s


In [28]:
python = pairs(answers, 'D:\PROJECT\dataset_select\y\python.csv')

ParserError: Expected 4 fields in line 10, saw 5